In [ ]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

# Configuração do Selenium para rodar no GitHub Actions
options = Options()
options.add_argument("--headless")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


# 🔹 Função para coletar os jogos sem acessar individualmente
def collect_games():
    print("🔄 Coletando lista de jogos...")
    
    # Iniciar Selenium para buscar os jogos
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Acessar a página da Steam
    URL = "https://store.steampowered.com/search/?filter=popularcomingsoon&ndl=1&cc=br"
    driver.get(URL)
    time.sleep(3)

    # 🔹 Rolando a página para carregar todos os jogos
    last_height = driver.execute_script("return document.body.scrollHeight")
    retries = 0
    while retries < 10:  # Limite para evitar loops infinitos
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Espera para carregar mais jogos
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            retries += 1
        else:
            retries = 0
        last_height = new_height

    print("✅ Todos os jogos carregados!")

    # 🔹 Coletando informações básicas dos jogos
    games_data = []
    games = driver.find_elements(By.CSS_SELECTOR, ".search_result_row")  # Seleciona os jogos

    print(f"🔄 Coletando {len(games)} jogos...")

    for game in tqdm(games, desc="📥 Coletando informações dos jogos", unit=" jogo"):
        title_elem = game.find_element(By.CSS_SELECTOR, ".title")
        release_date_elem = game.find_element(By.CSS_SELECTOR, ".search_released")
        
        # 🔹 Captura do preço
        price_elem = game.find_elements(By.CSS_SELECTOR, ".discount_final_price")  # Preço com desconto
        if not price_elem:
            price_elem = game.find_elements(By.CSS_SELECTOR, ".search_price")  # Preço sem desconto
        price = price_elem[0].text.strip() if price_elem else "Indefinido"

        # 🔹 Captura da URL do jogo
        game_url = game.get_attribute("href")

        # 🔹 Captura do título e data de lançamento (fallback para evitar valores vazios)
        title = title_elem.text.strip() if title_elem.text.strip() else "Título não disponível"
        release_date = release_date_elem.text.strip() if release_date_elem.text.strip() else "Data não informada"

        games_data.append({
            "title": title,
            "release_date": release_date,
            "price": price,
            "game_url": game_url,
            "genres": "A capturar..."  # Placeholder para depois capturar os gêneros
        })

    driver.quit()  # Fecha o Selenium
    return games_data


def get_game_details(game_url):
    try:
        # 🔹 Configuração do Selenium
# 🔹 Inicializar Selenium apenas UMA vez para a segunda etapa
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("🔄 Capturando informações adicionais via Selenium...")

for game in tqdm(games, desc="📂 Coletando detalhes dos jogos", unit=" jogo"):
    if game["game_url"] == "URL não disponível":
        game["genres"] = "Sem gênero"
        game["price"] = "Indefinido"
    else:
        try:
            # 🔹 Acessa a página com `cc=br` para garantir BRL
            game_url = game["game_url"]
            if "cc=" not in game_url:
                game_url += "?cc=br"

            driver.get(game_url)

            # 🔹 Espera o carregamento do título
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".apphub_AppName"))
            )

            # 🔹 Capturar título e data de lançamento
            title_elem = driver.find_element(By.CSS_SELECTOR, ".apphub_AppName")
            release_date_elem = driver.find_element(By.CSS_SELECTOR, ".release_date .date")

            game["title"] = title_elem.text.strip() if title_elem else "Título não disponível"
            game["release_date"] = release_date_elem.text.strip() if release_date_elem else "Data não informada"

            # 🔹 Capturar os gêneros corretamente
            genre_elems = driver.find_elements(By.CSS_SELECTOR, ".glance_tags.popular_tags a")
            game["genres"] = ", ".join([g.text.strip() for g in genre_elems]) if genre_elems else "Sem gênero"

            # 🔹 Capturar o preço corretamente (em reais)
            price_elem = driver.find_elements(By.CSS_SELECTOR, ".game_purchase_price, .discount_final_price")
            game["price"] = price_elem[0].text.strip() if price_elem else "Indefinido"

        except Exception as e:
            game["genres"] = "Erro ao capturar"
            game["price"] = "Indefinido"

# 🔹 Fechar o Selenium após o loop
driver.quit()

# 🔹 Salvar os dados em CSV
df = pd.DataFrame(games)
df.to_csv("steam_upcoming_games.csv", index=False, encoding="utf-8")

print(f"✅ Coletados {len(games)} jogos futuros e salvos em steam_upcoming_games.csv")

